In [2]:
import pandas as pd

df21 = pd.read_csv("/opt/conda/anaconda/lib/python3.6/site-packages/irsx/CSV/index_2021.csv",index_col=False)
object_ids = df21["OBJECT_ID"].to_numpy()

/opt/conda/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
object_ids_s = object_ids[:100]
object_ids_s

array([202010659349301301, 202010729349300041, 202010729349300021,
       202010729349300836, 202000669349300135, 202000669349301125,
       202000669349301320, 202000669349301850, 202000669349300905,
       202000669349300700, 202000669349300425, 202000679349300505,
       202000679349300405, 202000679349300210, 202020669349300127,
       202000729349300710, 202020689349300402, 202030699349301268,
       202000699349300330, 202000699349300740, 202010679349300301,
       202020699349301072, 202020699349300617, 202030709349300313,
       202000709349301455, 202000709349300200, 202000709349300520,
       202010699349301266, 202010699349300541, 202000719349301035,
       202010709349300601, 202010709349301231, 202010709349300931,
       202010709349301506, 202020719349300727, 202010719349300816,
       202010719349301566, 202010719349301446, 202040699349300954,
       202040669349300604, 202040709349301429, 202040719349301204,
       202020589349300437, 202020589349300007, 202020589349301

In [4]:
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
list_all = []
for e in object_ids_s:
    filing = xml_runner.run_filing(e)
    schedules = filing.list_schedules()
    
    ein = 0
    state = 0
    name = 0
    revenue = 0
    revenueEZ = 0
    
    if "ReturnHeader990x" in schedules:
        header = filing.get_parsed_sked("ReturnHeader990x")
        #print(len(header))
        header_part_i = header[0]["schedule_parts"]["returnheader990x_part_i"]
        #print(header_part_i)
        ein = header_part_i["ein"]
        try:
            state = header_part_i["USAddrss_SttAbbrvtnCd"]
        except KeyError:
            state = XX
        name = header_part_i["BsnssNm_BsnssNmLn1Txt"]
        #print(ein, state, name)
        
    if "IRS990EZ" in schedules:
        irs990ez = filing.get_parsed_sked("IRS990EZ")
        irs990ez_part_i = irs990ez[0]["schedule_parts"]["ez_part_i"]
        revenueEZ = irs990ez_part_i["TtlRvnAmt"]        
    
    if "IRS990" in schedules:
        irs990 = filing.get_parsed_sked("IRS990")
        #print(len(irs990))
        irs990_part_i = irs990[0]["schedule_parts"]["part_i"]
        revenue = irs990_part_i["CYTtlRvnAmt"]
        #print(revenue)
        
    list = [e, ein, state, name, revenue, revenueEZ]
    list_all.append(list)
    
df_result = pd.DataFrame(list_all, columns =['ObjectID', 'EIN', 'State', 'Name', 'Revenue', 'RevenueEZ'])
df_result.to_csv('revenue_2021_100.csv', index = False)
df_result
  

        
    #print(e, ein, state, name, revenue, revenueEZ)

,ObjectID,EIN,State,Name,Revenue,RevenueEZ
0,202010659349301301,452772761,NJ,CAMDEN'S CHARTER SCHOOL,5148336,0
1,202010729349300041,237061115,MS,JACKSON STATE UNIVERSITY,10386900,0
2,202010729349300021,344427516,OH,TIFFIN UNIVERSITY,64789812,0
3,202010729349300836,840865247,CO,NETWORK MINISTRIES INC,218029,0
4,202000669349300135,205647589,SC,FAMILY PROMISE OF BEAUFORT COUNTY,482668,0
...,...,...,...,...,...,...
95,202021379349100307,412209842,GA,LOVE PEACE AND PROSPERITY INTERNATIONAL INC,0,0
96,202021379349100602,136216916,FL,SIMON AND STELLA SHEIB FOUNDATION,0,0
97,202021379349200022,202272015,SC,ALPHA SIGMA PHI FRATERNITY INC,0,198372
98,202021379349200512,431336280,MO,MISSOURI DAIRY ASSOCIATIONINC,0,94662


## Start a Sparkession

In [5]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext(appName="BigDataIRS2")
spark = SparkSession(sc)

## Einlesen des Files  
Das File kann entweder lokal oder mittels hdfs eingelesen werden  
Falls das File lokal eingelesen wird muss es auf jedem Node vorhanden sein, deswegen empfiehlt sich die Verwendung von HDFS 

In [ ]:
fs_path = "file:///spark_csv/revenue_2021_100.csv" #local_fs
hdfs_path = "hdfs://spark-jupyter-m/user/hdfs/spark_csv/revenue_2021_100.csv" # hdfs
df = spark.read.csv(hdfs_path, header=True, inferSchema = True)

type(df), df.printSchema(), df.head(5)

## Simple Aggregation
Das DF kann entweder direkt aggregiert werden, oder vorher in ein RDD umgewandelt werden

In [1]:
# Aggregation als DF
grouped_df = df.groupby("State").sum("Revenue").sort("Sum(Revenue)", ascending = False)

# Aggregation als RDD
rdd = df.rdd.drop("ObjectID", "EIN", "Name", "RevenueEZ") # U
from operator import add
reduced_rdd = rdd.reduceByKey(add).sortBy(lambda x: x[1], ascending = False)
reduced_rdd.collect()

NameError: name 'df' is not defined

In [ ]:
#from google.cloud import storage
#client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
#bucket = client.get_bucket('sparkbucket02')
# Then do other things...
#blob = bucket.get_blob("revenue_2021_100.csv") #('remote/path/to/file.txt')

#df = pd.read_csv(blob.download_as_string())
#df

In [2]:
#spark = SparkSession \
#    .builder \
#    .appName("Protob Conversion to Parquet") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()\

#df = spark.read.csv('/home/hadoop/observations_temp.csv, header=True)

sudo su - hdfs  
hdfs dfsadmin -safemode leave  

hdfs dfs -mkdir spark_csv  
hdfs dfs -put /spark_csv/revenue_2021_100.csv spark_csv/revenue_2021_100.csv  
hdfs dfs -ls spark_csv  

https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
https://stackoverflow.com/questions/33055403/how-to-navigate-directories-in-hadoop-hdfs
https://stackoverflow.com/questions/28213116/hadoop-copy-a-local-file-system-folder-to-hdfs
https://stackoverflow.com/questions/61197811/can-i-read-csv-files-from-google-storage-using-spark-in-more-than-one-executor
https://groups.google.com/g/cloud-dataproc-discuss/c/cubkWrjkk2g?pli=1
https://stackoverflow.com/questions/56448009/storing-source-file-in-google-dataproc-hdfs-vs-google-cloud-storagegoogle-bucke

hdfs dfs -put /spark_csv/revenue_2021_100.csv /user/root/spark_csv/revenue_2021_100.csv